In [ ]:
import kagglehub
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import mixed_precision
import pathlib

mixed_precision.set_global_policy('mixed_float16')

path = kagglehub.dataset_download("jessicali9530/stanford-dogs-dataset")
dataset_path = pathlib.Path(path) / "images" / "Images"

image_height = 240
image_width = 240
batch_size = 32


train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path, validation_split=0.2, subset="training", seed=123,
    image_size=(image_height, image_width), batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path, validation_split=0.2, subset="validation", seed=123,
    image_size=(image_height, image_width), batch_size=batch_size
)

model_path = "dog_spotter_model.keras"


AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.shuffle(200).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

num_classes = 120

model = keras.models.Sequential([
    keras.layers.Rescaling(1./255, input_shape=(image_height, image_width, 3)),
    keras.layers.Conv2D(32, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(64, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(num_classes, activation="softmax", dtype='float32')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
    jit_compile=True
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=500
)

Using Colab cache for faster access to the 'stanford-dogs-dataset' dataset.
Creating Data Pipeline... (Usage: Low RAM)
Found 20580 files belonging to 120 classes.
Using 16464 files for training.
Found 20580 files belonging to 120 classes.
Using 4116 files for validation.
Starting Training... (Usage: GPU kicks in now)
Epoch 1/500
515/515 ━━━━━━━━━━━━━━━━━━━━ 50s 67ms/step - accuracy: 0.0094 - loss: 4.7874 - val_accuracy: 0.0104 - val_loss: 4.7842
Epoch 2/500
515/515 ━━━━━━━━━━━━━━━━━━━━ 42s 61ms/step - accuracy: 0.0128 - loss: 4.7826 - val_accuracy: 0.0141 - val_loss: 4.7708
Epoch 3/500
515/515 ━━━━━━━━━━━━━━━━━━━━ 82s 62ms/step - accuracy: 0.0173 - loss: 4.7585 - val_accuracy: 0.0190 - val_loss: 4.7322
Epoch 4/500
515/515 ━━━━━━━━━━━━━━━━━━━━ 40s 60ms/step - accuracy: 0.0155 - loss: 4.7128 - val_accuracy: 0.0182 - val_loss: 4.6854
Epoch 5/500
515/515 ━━━━━━━━━━━━━━━━━━━━ 42s 59ms/step - accuracy: 0.0194 - loss: 4.6761 - val_accuracy: 0.0224 - val_loss: 4.6518
Epoch 6/500
515/515 ━━━━━━